In [38]:
import numpy as np
import PIL
import matplotlib.pyplot as plt
import os
import math

In [4]:
base_dir = "/notebooks/TugasAkhir/Data/skin-90"

## Load Data

In [21]:
def load_data(directory, image_size = (256, 256)) :
    
    total_image = 0
    #get folder/class name in dataset folder
    class_name_list = [x.lower() for x in os.listdir(directory) if os.path.isdir(os.path.join(base_dir, x))]
    for class_name in class_name_list :
        total_image += len(os.listdir(os.path.join(base_dir,class_name)))
    
    stack_of_image = np.empty((total_image,image_size[0],image_size[1],3), dtype = np.uint8)
    stack_of_label = np.empty(total_image, dtype = "S{}".format(len(max(class_name_list))))

    # Read Image and Class then append to array
    temp = total_image 
    for class_name in class_name_list :
        image_class_path = os.path.join(directory, class_name)
        for image_name in os.listdir(image_class_path) :
            image_path = os.path.join(image_class_path,image_name)
            
            # Open Image 
            image = PIL.Image.open(image_path)
            
            # Convert Image to RGB
            image = image.convert("RGB")
            
            # Resize             
            image = image.resize(image_size, resample = PIL.Image.Resampling.BICUBIC)
            
            #Get Array Of the Image
            image_array = np.array(image)
            
            #Append to Array
            stack_of_image[total_image-temp] = image_array
            stack_of_label[total_image-temp] = class_name
            temp -= 1
            
    return stack_of_image,stack_of_label


In [22]:
(X, y) = load_data(base_dir)

## Train Test Split

In [61]:
def train_test_split(X,y, test_size= 0.2):
    train_index = []
    class_name, start_index_class,  count_each_class = np.unique(y, return_index = True,return_counts = True)
    
    train_size= 1-test_size
    
    num_of_train = math.ceil(np.min(count_each_class) * train_size)
    
    for start_index in start_index_class :
        train_index += [*range(start_index, start_index+num_of_train)]
    
    test_index = [x for x in range(0,len(y)) if x not in train_index]
    
    # Shuffle the index so the array not sorted by label
    np.random.shuffle(train_index)
    np.random.shuffle(test_index)
    
    return X[train_index], X[test_index], y[train_index], y[test_index]
    
    

In [59]:
X_train,X_test,y_train, y_test = train_test_split(X,y)

## Modeling with Keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    rotation_range = 30,
    zoom_range = 0.3,
    validation_split = 0.2)

In [1]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(256, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

NameError: name 'tf' is not defined

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
history = model.fit(datagen.flow(X_train, y_train,subset='training',batch_size = 128),
         validation_data=datagen.flow(X_train, y_train, subset='validation',batch_size = 1),epochs=10)